In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import regionmask
from shapely import MultiPolygon, Polygon
from shapely.affinity import rotate

from tams.shapes import *

## Shapes

### Line

In [ ]:
make_line((0, 0), (1, 2))

### Arc

In [ ]:
display(
    make_arc((0, 0), 1, (150, 210.1)),
    make_arc((0, 0), 1, (-25, 25)),
    make_arc((0, 0), 1, (25, -25)),
)

### Rectangle

In [ ]:
rect = make_rectangle((0, 0), (4, 2))

display(rect)

gpd.GeoSeries(rect).plot()

In [ ]:
display(
    make_rectangle2((0, 0), 3, 5),
    make_square((0, 0), 1),
    rotate(make_square((0, 0), 1), 30),
)

### Circle

In [ ]:
cir = make_circle((0, 0), 1)

assert cir.bounds[2] - cir.bounds[0] == 2

cir

In [ ]:
knife = make_line((-1, -5), (1, 5))
display(knife)

polys = split(cir, knife)
assert len(polys) == 2

gpd.GeoSeries(polys).plot(cmap="tab10")

### Ellipse

In [ ]:
ell = make_ellipse((0, 0), 1, 1, angle=0)
assert ell.bounds == (-0.5, -0.5, 0.5, 0.5)

ell = make_ellipse((0, 0), 1, 1, angle=0, half=True)
assert ell.bounds == (-1, -1, 1, 1)

ell = make_ellipse((-10, 5), 1.2, 0.8, angle=15)

display(ell)

gpd.GeoSeries(ell, crs="EPSG:4326").plot(fc="none")

### Cone ("circle-cone")

In [ ]:
x_range = (0, -10)
y_range = (0, 0)
r_range = (0.2, 3)

cone = make_cone(x_range, y_range, r_range)

display(cone)

gpd.GeoSeries(cone, crs="EPSG:4326").plot(fc="none")


polys = split(cone, make_arc((-3, 0), 4, (45, 360 - 45)))
assert len(polys) == 2

display(MultiPolygon(polys))

gpd.GeoSeries(polys).plot(cmap="tab10")

### Cone (capped with arc)

In [ ]:
display(make_cone2((0, 0), 10, 5, rcap=np.inf))  # flat (triangle)
display(make_cone2((0, 0), 10, 5))  # circle the cone is based on
display(make_cone2((0, 0), 10, 5, rcap=5))  # more curved
display(make_cone2((0, 0), 10, 5, rcap=2.5))  # like the circle-cone

In [ ]:
# Something to think about is making the shape in the projection
# and then translating back to lat/lon
import matplotlib.pyplot as plt

from pyproj import Transformer

# crs = "EPSG:32663"  # equidistant cylindrical (no dlon change with lat; does nothing to the shape)
# crs = "EPSG:3857"  # web mercator (no dlon change with lat, but cap distorts at higher lats)
# crs = "EPSG:7789"  # this one seems slow
# crs = "EPSG:3035"  # Gall-Peters (equal-area)
# crs = "+proj=laea"  # Lambert Azimuthal equal-area
crs = "+proj=laea +lon_0=0 + lat_0=80"
# crs = "+proj=aeqd"  # Azimuthal Equidistant (should choose center pt)
# crs = "+proj=aeqd +lon_0=0 +lat_0=80"  # Azimuthal Equidistant
latlon = "EPSG:4326"

print(gpd.GeoSeries(make_rectangle2((0, 80), 5, 2), crs=latlon).to_crs("EPSG:32663").area)

tran = Transformer.from_crs(latlon, crs, always_xy=True)

x = 0
y = 80
h = 5
d = 0.6 * h
xy_crs = tran.transform(x, y)
h_crs = tran.transform(x + h, y)[0] - xy_crs[0]
d_crs = tran.transform(x, y + d)[1] - xy_crs[1]

gpd.GeoSeries(
    make_cone2(xy_crs, h_crs, d_crs),
    crs=crs,
).to_crs(latlon).plot()
plt.gca().set_aspect("equal", "box");

## Examples

### Multiple cones

e.g. to apply to a moving CE or MCS centroid at different times

In [ ]:
x = np.arange(5)
y = -0.5 * x
cen = gpd.GeoSeries(gpd.points_from_xy(x, y), crs="EPSG:4326")

dx, dy, r_range = 7, 0, (0.5, 2.5)

cones = cen.apply(lambda p: make_cone((p.x, p.x + dx), (p.y, p.y + dy), r_range))
cones.plot(fc="none")

### Quadrants

In [ ]:
from itertools import chain

xy = (0, 0)

angle = -10  # ellipse rotation (deg)

assert -45 < angle < 45
outer = make_ellipse(xy, w=7, h=5.5, angle=angle, half=True)
inner = make_ellipse(xy, w=3, h=2.5, angle=angle, half=True)

fb_angle = 120  # size (angle) of the backward/forward-pointing sections (deg)
rotate = -5  # rotate cuts wrt. horizontal (positive -> counterclockwise; deg)
r = 1.1 * 7

assert 0 < abs(fb_angle) < 180
assert -45 < rotate < 45
angle2 = 180 - fb_angle
t = np.deg2rad(rotate + fb_angle/2 + np.r_[0, angle2, angle2 + fb_angle, fb_angle + 2 * angle2])
x0, x1, x2, x3 = r * np.cos(t)
y0, y1, y2, y3 = r * np.sin(t)

line1 = make_line((x2, y2), (x0, y0))
line2 = make_line((x1, y1), (x3, y3))

cutters = [inner, line1, line2]

polys = [outer]
for cutter in cutters:
    polys = list(chain.from_iterable(split(p, cutter) for p in polys))
# assert len(polys) == 8

s = gpd.GeoSeries(polys)
assert np.isclose(s.area.sum(), outer.area)

to_long = {
    "I": "inner",
    "O": "outer",
    "R": "right",
    "L": "left",
    "D": "down",
    "U": "up",
}
abbrevs = []
names = []
for io in "IO":
    for d in "RDLU":
        abbrevs.append(f"{io}{d}")
        names.append(f"{to_long[io]} {to_long[d]}")

df = s.to_frame(name="geometry").assign(abbrev=abbrevs, name=names)
display(df)

df.plot(cmap="tab10", column="abbrev", legend=True)

You may wish to exclude the CE/MCS area.
It can be easily subtracted from the {class}`~geopandas.GeoSeries`:

In [ ]:
ce = make_ellipse((0, 0), 1.6, 0.9, angle=angle)
df_ = df.assign(geometry=df.difference(ce))
df_.plot(cmap="tab10", column="abbrev", legend=True)

Or included as an additional region:

In [ ]:
df_ = df.assign(geometry=df.difference(ce))
df_ = pd.concat(
    [
        df_,
        gpd.GeoSeries(ce).to_frame(name="geometry").assign(abbrev="CE", name="cloud element")
    ],
    ignore_index=True,
)
df_.plot(cmap="tab10", column="abbrev", legend=True)

In [ ]:
regions = regionmask.from_geopandas(df_, abbrevs="abbrev", names="name")
display(regions)
regions.plot()

### Forward and backward cones

In [ ]:
from itertools import chain

xc, yc = (5, 5)

h = 10  # to center of final circle
r = 7  # radius of final circle
rotate = -2
rmin = 0.35

assert -45 < rotate < 45
r_range = (rmin, r)
r = np.r_[rmin, h]
t = np.deg2rad(rotate + np.r_[0, 180])
x_range1, x_range2 = xc + np.outer(np.cos(t), r)
y_range1, y_range2 = yc + np.outer(np.sin(t), r)

c1 = make_cone(x_range1, y_range1, r_range)
c2 = make_cone(x_range2, y_range2, r_range)

polys = [c1, c2]

# Smooth the outer edges to a circle arc
cutter = make_circle((xc, yc), 1.1 * h)
polys = [p.intersection(cutter) for p in polys]

# Split using a smaller circle
cutter = make_circle((xc, yc), 0.5 * h).boundary
polys = list(chain.from_iterable(split(p, cutter) for p in polys))

# display(*polys)
# display(MultiPolygon(polys))

abbrevs = []
names = []
for d in "RL":
    for io in "IO":
        abbrevs.append(f"{io}{d}")
        names.append(f"{to_long[io]} {to_long[d]}")

df = gpd.GeoSeries(polys).to_frame(name="geometry").assign(abbrev=abbrevs, name=names)
display(df)

df.plot(cmap="tab10", column="abbrev", legend=True)